### Structured Output
- "Labrador is a breed of Dog"
- - { 'content':'Dog', 'species':'Labra' }

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage

In [3]:
from pydantic import BaseModel

class PlanetInfo(BaseModel):
    name: str
    color: str
    distance_miles: int

#### Unstrucuted Output Provided Output Based On System Message ??

In [4]:
unstructured_model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash",
    api_key=os.getenv("GEMINI_API_KEY"),
)

In [5]:
agent = AssistantAgent(
    name="planet_agent",
    model_client=unstructured_model_client,
    system_message="You are a helpful assistant that provides information about planets. in the structure JSON" \
    "{ name :str" \
    "age : int" \
    "}"
)

In [6]:
async def test_unstructured_output():
    task = TextMessage(content="Please provide information about Mars.", source="user")
    
    result = await agent.run(task=task)
    
    unstructured_response = result.messages[-1].content
    
    print(unstructured_response)

await test_unstructured_output()

```json
{
  "name": "Mars",
  "surface_temperature": {
    "average": -63,
    "min": -140,
    "max": 20 
  },
  "gravity": 3.71,
  "diameter": 6779,
  "mass": 6.42e23,
  "moons": ["Phobos", "Deimos"],
  "atmosphere": {
    "composition": {
      "carbon_dioxide": 95.32,
      "nitrogen": 2.7,
      "argon": 1.6,
      "oxygen": 0.13,
      "others": 0.25
    },
    "pressure": 0.006
  },
  "orbital_period": 687,
  "rotation_period": 24.6,
  "axial_tilt": 25.19
}
```



#### Structed Output With Pydantic Class We Created ??

In [7]:
model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash",
    api_key=os.getenv("GEMINI_API_KEY"),
    response_format=PlanetInfo
)

In [8]:
agent = AssistantAgent(
    name="planet_agent",
    model_client=model_client,
    system_message="You are a helpful assistant that provides information about planets. in the structure JSON"
)

In [9]:
async def test_structured_output():
    task = TextMessage(content = "Please provide information about Mars.", source='User')
    
    result = await agent.run(task=task)
    
    structured_response = result.messages[-1].content
    
    print(structured_response)

await test_structured_output()

{"color": "red", "distance_miles": 140000000, "name": "Mars"}
